In [1]:
import os
import json 

# Change current path in the system 
os.chdir('/home/ad/RoutingComparasion/RoutingComparation/routingcomparison')

# # Check the current directory 
# current_directory = os.getcwd()
# print("Standing at:", current_directory)
def read_data(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

from routingapp.compare_algorithm.sec_morl_multipolicy.train import train_sdn_policy
from routingapp.compare_algorithm.sec_morl_multipolicy.module_function import Function
from routingapp.compare_algorithm.sec_morl_multipolicy.module_graph import Graph
from routingapp.compare_algorithm.sec_morl_multipolicy.function_dijkstra import dijkstra, routing_k


# DFS function to find paths from source to destination
def dfs(graph, start, goal):
    visited = set()  # List of visited nodes
    stack = [(start, [start])]  # Stack containing pairs (node, path from source to node)

    while stack:
        node, path = stack.pop()  # Get the last node from the stack and the path to it
        if node not in visited:
            visited.add(node)
            if node == goal:
                return path  # Return the path from source to destination
            neighbors = graph.adj_matrix[node]  # Access neighbors from the adjacency matrix of the graph
            for neighbor in reversed(neighbors):  # Traverse neighbors in reverse order to use stack
                if neighbor not in visited:
                    stack.append((neighbor, path + [neighbor]))  # Add unvisited neighbors to stack with path to that node

    return None  # If no path from source to destination is found


# Main function 
def run_sec(data_path, result_store):
    # Read data from JSON file
    data = read_data(data_path)
    nodes = data['nodes']
    edge_servers = data['edge_server']
    cloud_servers = data['cloud_server']
    clients = data['client']
    edges = data['edges']
    adj_matrix = [[] for _ in range(len(nodes))]
    for edge in edges:
        adj_matrix[edge[0]].append(edge[1])
        adj_matrix[edge[1]].append(edge[0])

    # Define the graph object
    graph = Graph(len(nodes), len(clients), len(edge_servers), len(cloud_servers), len(clients), clients, edge_servers, cloud_servers, adj_matrix)

    # Initialize the function object
    func = Function()

    # Iterate through each scenario in the data
    scenario_list = data['scenario']
    for scenario in scenario_list:
        sum_delay_topo = 0
        delay_update = scenario["delay"]
        for delay in delay_update:
            sum_delay_topo  = sum_delay_topo + delay[2]
        bandwidth_update = scenario["bandwidth"]
        loss_update = scenario["loss"]
        request = scenario["request"]

        # Update the graph with delay, loss, and bandwidth information
        graph.updateGraph(delay_update, loss_update, bandwidth_update)

        print(graph.adj_matrix, graph.number_nodes, graph.number_edge_servers, graph.number_clients, graph.number_cloud_servers)
        # Generate the promising paths using DFS 
        promising_paths = []
        # print(request)
        for src, dst in request:
            print(src, dst)
            path = dfs(graph, src, dst)
            if path:
                promising_paths.append(path)
            print(promising_paths)

            # Create a subgraph containing all paths in the promising paths     
            promising_nodes = set()
            for path in promising_paths:
                print(path)
                promising_nodes.update(path)
            print(promising_nodes)
            promising_graph = graph.subgraph(list(promising_nodes))


            # Update src and dst in the request to match the new node indices
            promising_nodes_list = list(promising_nodes)

            # Update src and dst in the request to match the new node indices
            request = [(promising_nodes_list.index(src), promising_nodes_list.index(dst))]
            
            print(request)
            print(promising_graph.adj_matrix, promising_graph.number_nodes, promising_graph.number_edge_servers, promising_graph.number_clients, promising_graph.number_cloud_servers)
        

        # Train the DRL model using the promising graph and the function object
        trained_models = train_sdn_policy(promising_graph, func, request[0])
        
        if (trained_models == None):
            print("No trained models")
            return None
        else 
        # Use the trained models to generate solutions
        for req in request: 
            print(req)
            solutions = func.generate_solutions(promising_graph, func, req)  
        
            # Select the best solution
            result = func.select_solution(solutions)

    # Write the results to a JSON file
    with open(result_store, 'w') as f:
        json.dump(result, f)


In [2]:
run_sec(r"/home/ad/RoutingComparasion/RoutingComparation/routingcomparison/routingapp/compare_algorithm/data/chinanet/2_edge_cloud_server.json",r'result\sec_solver\chinanet_2_edge_cloud_server.json')

[[16, 3, 39], [18, 39], [33], [0], [8], [38], [18, 39], [39], [4, 38, 39, 9, 11, 16, 18, 23, 24, 25, 26, 27, 28, 31], [8, 27], [39], [8], [28], [25], [16, 28], [16, 28], [0, 8, 14, 15, 27, 28], [28], [1, 6, 8, 32, 33, 41, 39, 40, 25, 27, 28], [39], [23], [28], [25, 28], [8, 20, 28, 39], [8], [8, 13, 18, 22, 39, 27], [8], [8, 9, 16, 18, 25, 39, 30], [8, 12, 14, 15, 16, 17, 18, 21, 22, 23, 38, 39, 29], [28], [27], [8], [18, 39], [2, 18, 39], [39], [39], [39], [38], [5, 8, 28, 37, 39], [0, 1, 6, 7, 8, 10, 18, 19, 23, 25, 27, 28, 32, 33, 34, 35, 36, 38, 40, 41], [18, 39], [18, 39]] 42 1 40 1
13 28
[[13, 25, 8, 38, 28]]
[13, 25, 8, 38, 28]
{38, 8, 13, 25, 28}
[(2, 4)]
[[5, 8, 28, 37, 39], [4, 38, 39, 9, 11, 16, 18, 23, 24, 25, 26, 27, 28, 31], [25], [8, 13, 18, 22, 39, 27], [8, 12, 14, 15, 16, 17, 18, 21, 22, 23, 38, 39, 29], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

/home/ad/RoutingComparasion/venv11/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/ad/RoutingComparasion/venv11/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Edge Action: 0, Cloud Action: -1
[28] 28
[0] 0
dist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[5]
dist [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
